# *Unsupervised learning: Latent Dirichlet allocation (LDA) topic modeling*

In [1]:
## Install Python package for LDA
# http://pythonhosted.org/lda/getting_started.html

!pip3 install lda

    100% |████████████████████████████████| 512kB 956kB/s ta 0:00:011
    100% |████████████████████████████████| 102kB 3.1MB/s a 0:00:01


In [2]:
## Importing basic packages

import os
import numpy as np

In [3]:
os.chdir('/sharedfolder/')

!wget https://github.com/pcda17/pcda17.github.io/raw/master/week/10/nyt_articles_11-9-2017.zip

!unzip nyt_articles_11-9-2017.zip

os.chdir('/sharedfolder/nyt_articles_11-9-2017/')

--2017-11-10 22:26:15--  https://github.com/pcda17/pcda17.github.io/raw/master/week/10/nyt_articles_11-9-2017.zip
Resolving github.com (github.com)... 192.30.253.113, 192.30.253.112
Connecting to github.com (github.com)|192.30.253.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/pcda17/pcda17.github.io/master/week/10/nyt_articles_11-9-2017.zip [following]
--2017-11-10 22:26:16--  https://raw.githubusercontent.com/pcda17/pcda17.github.io/master/week/10/nyt_articles_11-9-2017.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1172825 (1.1M) [application/zip]
Saving to: ‘nyt_articles_11-9-2017.zip’

nyt_articles_11-9-2 100%[===================>]   1.12M  --.-KB/s    in 0.1s    

2017-11-10 22:26:16 (7.65 MB/s) - ‘nyt_articles_11-9-20

  inflating: nyt_articles_11-9-2017/destination-unknown-review.txt  
  inflating: nyt_articles_11-9-2017/detroit-pistons-nba-little-caesars.txt  
  inflating: nyt_articles_11-9-2017/diana-nyad-sexual-assault.txt  
  inflating: nyt_articles_11-9-2017/dinosaurs-mammals-daytime.txt  
  inflating: nyt_articles_11-9-2017/dior-sidney-toledano-pietro-beccari.txt  
  inflating: nyt_articles_11-9-2017/do-i-have-to-allow-a-private-investigator-into-my-apartment.txt  
  inflating: nyt_articles_11-9-2017/dogs-argentine-ants.txt  
  inflating: nyt_articles_11-9-2017/donald-trump-china-trade-xi-jinping.txt  
  inflating: nyt_articles_11-9-2017/donald-trump-democratic-party-att.txt  
  inflating: nyt_articles_11-9-2017/donald-trump-impersonator.txt  
  inflating: nyt_articles_11-9-2017/dont-nudge-me-the-limits-of-behavioral-economics-in-medicine.txt  
  inflating: nyt_articles_11-9-2017/dries-van-noten-polaroids.txt  
  inflating: nyt_articles_11-9-2017/dry-skin-winter-protection.txt  
  inflating: n

  inflating: nyt_articles_11-9-2017/looking-for-surprises-in-senegal.txt  
  inflating: nyt_articles_11-9-2017/lost-landscapes-of-new-york.txt  
  inflating: nyt_articles_11-9-2017/lp5558.txt  
  inflating: nyt_articles_11-9-2017/lung-cancer.txt  
  inflating: nyt_articles_11-9-2017/lvmh-stephanie-watine-arnault-clos19.txt  
  inflating: nyt_articles_11-9-2017/mabou-mines-and-target-margin-theater-get-permanent-homes.txt  
  inflating: nyt_articles_11-9-2017/macys-earnings.txt  
  inflating: nyt_articles_11-9-2017/made-in-china-technology-trade.txt  
  inflating: nyt_articles_11-9-2017/mag-job-at-war.txt  
  inflating: nyt_articles_11-9-2017/maher-zain-tour.txt  
  inflating: nyt_articles_11-9-2017/maine-medicaid-hospitals.txt  
  inflating: nyt_articles_11-9-2017/makie-yahagi-inspiration.txt  
  inflating: nyt_articles_11-9-2017/mammoth-fossils-males.txt  
  inflating: nyt_articles_11-9-2017/margaritas-review-dominican-food-bronx-alcapurrias.txt  
  inflating: nyt_articles_11-9-2017/m

  inflating: nyt_articles_11-9-2017/rory-smith.txt  
  inflating: nyt_articles_11-9-2017/roy-halladay-dead-plane.txt  
  inflating: nyt_articles_11-9-2017/rule-would-push-prosecutors-to-release-evidence-favorable-to-defense.txt  
  inflating: nyt_articles_11-9-2017/russia-doping-winter-olympics.txt  
  inflating: nyt_articles_11-9-2017/russia-election-reality-tv.txt  
  inflating: nyt_articles_11-9-2017/russia-venezuela-debt.txt  
  inflating: nyt_articles_11-9-2017/salvador-minuchin-a-pioneer-of-family-therapy-dies-at-96.txt  
  inflating: nyt_articles_11-9-2017/sam-dastyari-senator-racist.txt  
  inflating: nyt_articles_11-9-2017/saudi-arabia-crown-prince-arrests-iran.txt  
  inflating: nyt_articles_11-9-2017/saudi-arabia.txt  
  inflating: nyt_articles_11-9-2017/saudi-prince-reform-coup.txt  
  inflating: nyt_articles_11-9-2017/sen-sakana-review-peruvian-food-nikkei.txt  
  inflating: nyt_articles_11-9-2017/senate-confirmation-climate-epa.txt  
  inflating: nyt_articles_11-9-2017/se

  inflating: nyt_articles_11-9-2017/wanted-a-corporate-kaepernick-mellody-hobson-on-workplace-diversity.txt  
  inflating: nyt_articles_11-9-2017/war-north-korea.txt  
  inflating: nyt_articles_11-9-2017/watch-kids-sing-stephen-sondheim-at-ts-greats-party.txt  
  inflating: nyt_articles_11-9-2017/waymo-autonomous-cars.txt  
  inflating: nyt_articles_11-9-2017/westchester-nassau-county-executive-results.txt  
  inflating: nyt_articles_11-9-2017/what-college-admissions-wants.txt  
  inflating: nyt_articles_11-9-2017/what-to-cook-this-week-newsletter.txt  
  inflating: nyt_articles_11-9-2017/what-to-cook-this-weekend-newsletter.txt  
  inflating: nyt_articles_11-9-2017/what-to-do-36-hours-in-kauai-hawaii.txt  
  inflating: nyt_articles_11-9-2017/what-to-do-36-hours-in-latino-new-york-city.txt  
  inflating: nyt_articles_11-9-2017/what-to-read-before-heading-to-the-caribbean.txt  
  inflating: nyt_articles_11-9-2017/whats-new-in-the-caribbean.txt  
  inflating: nyt_articles_11-9-2017/whats

In [4]:
document_list = []

for filename in [item for item in os.listdir('./') if '.txt' in item]:
    text_data = open(filename).read()
    document_list.append(text_data)

In [5]:
## Importing NLTK stop words
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
import string

 
stop_words = stopwords.words('english') + ["'s", "'re", '”', '“', '’', '—'] + list(string.punctuation)

string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [6]:
## Tokenizing and removing stop words from our list of documents

documents_filtered = []

for document in document_list:
    token_list = word_tokenize(document.lower())
    tokens_filtered = [item for item in token_list if (item not in stop_words)]
    documents_filtered.append(' '.join(tokens_filtered))

In [7]:
## Viewing a preprocessed document

documents_filtered[30]

'teachable robots mr. abbeel said recent interview new company offices emeryville calif. across bay san francisco new company part much wider effort create a.i allows robots learn researchers places like google brown university carnegie mellon similar work existing start-ups like micropsi prowler.io robots already automate work inside factories warehouses moving boxes place place amazon massive distribution centers companies must program machines particular task limiting possible applications hope robots master much wider array tasks learning today every motion industrial robot makes specified millimeter said sunil dhaliwal amplify founder led firm investment embodied intelligence real problems solved way able tell robot tell learn. photo mr. abbeel founders embodied intelligence including former openai researchers peter chen rocky duan former microsoft researcher tianhao zhang specialize algorithmic method called reinforcement learning way machines learn tasks extreme trial error rese

In [8]:
## Vectorizing preprocessed essays

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(documents_filtered) 

In [9]:
## Creating a vocabulary list corresponding to the vectors we created above

vocabulary = vectorizer.get_feature_names()

vocabulary[1140:1160]

['aftermath',
 'afternoon',
 'afterward',
 'again',
 'agata',
 'agate',
 'agatha',
 'age',
 'aged',
 'agena',
 'agence',
 'agencies',
 'agency',
 'agenda',
 'agendas',
 'agent',
 'agents',
 'ages',
 'agey',
 'agg']

In [10]:
## Initializing an LDA model: 10 topics and 1500 iterations

import lda

model = lda.LDA(n_topics=10, n_iter=1500, random_state=1)

In [11]:
## Fitting the model using our list of vectors

model.fit(X)

INFO:lda:n_documents: 451
INFO:lda:vocab_size: 25764
INFO:lda:n_words: 215633
INFO:lda:n_topics: 10
INFO:lda:n_iter: 1500
INFO:lda:<0> log likelihood: -2610850
INFO:lda:<10> log likelihood: -2148148
INFO:lda:<20> log likelihood: -2082446
INFO:lda:<30> log likelihood: -2050699
INFO:lda:<40> log likelihood: -2034686
INFO:lda:<50> log likelihood: -2025850
INFO:lda:<60> log likelihood: -2019056
INFO:lda:<70> log likelihood: -2012439
INFO:lda:<80> log likelihood: -2009303
INFO:lda:<90> log likelihood: -2005266
INFO:lda:<100> log likelihood: -2001732
INFO:lda:<110> log likelihood: -1999150
INFO:lda:<120> log likelihood: -1997936
INFO:lda:<130> log likelihood: -1995790
INFO:lda:<140> log likelihood: -1994592
INFO:lda:<150> log likelihood: -1992524
INFO:lda:<160> log likelihood: -1991903
INFO:lda:<170> log likelihood: -1989176
INFO:lda:<180> log likelihood: -1988396
INFO:lda:<190> log likelihood: -1989150
INFO:lda:<200> log likelihood: -1988587
INFO:lda:<210> log likelihood: -1987358
INFO:lda:

In [12]:
## Viewing the top 50 words in each 'topic'

topic_word = model.topic_word_

n_top_words = 100

for i, topic_distribution in enumerate(topic_word):
    topic_words = np.array(vocabulary)[np.argsort(topic_distribution)][:-(n_top_words+1):-1]
    print('Topic ' + str(i) + ':')
    print(' '.join(topic_words))
    print()

Topic 0:
like one people time many even years would new get way first still much back could make work world also want know say two long think well day part go need see something going called another around good made us right take photo reading life might really come last better every never though story often far things little three look less thought set put best find end away told help old got took days yet later without thing hard asked trying enough feel ago known point play whose others months use came found line home big great among went night

Topic 1:
food art fashion image wine red white cooking style made advertisement like continue club york restaurant thanksgiving main credit recipe look museum wines france though de new dinner chocolate table johnson flavors chef clothing styles chicken plastic bar photo butter van meal shop design paris french tables finger 2015 beer coffee best lakes arnold black fat collection taste menu eat cabernet daniel brand designer color meat chimp

### ▷Assignment

    Modify the code above: Apply a stemming step to each word before vectorizing the text.
    See example stemming code in the following cell.

In [ ]:
## Stemming example

from nltk.stem.porter import PorterStemmer

stemmer = PorterStemmer()

print(stemmer.stem('nature'))

print(stemmer.stem('natural'))

print(stemmer.stem('naturalism'))

# *Supervised learning: Naive Bayes classification*

In [ ]:
## Download sample text corpora from GitHub, then unzip.

os.chdir('/sharedfolder/')

## Uncomment the lines below if you need to re-download test corpora we used last week.

#!wget -N https://github.com/pcda17/pcda17.github.io/blob/master/week/8/Sample_corpora.zip?raw=true -O Sample_corpora.zip
#!unzip -o Sample_corpora.zip

In [ ]:
os.chdir('/sharedfolder/Sample_corpora')

os.listdir('./')

In [ ]:
## Loading Melville novels

os.chdir('/sharedfolder/Sample_corpora/Herman_Melville/')

melville_texts = []

for filename in os.listdir('./'):
    text_data = open(filename).read().replace('\n', ' ')
    melville_texts.append(text_data)

print(len(melville_texts))

In [ ]:
## Loading Austen novels

os.chdir('/sharedfolder/Sample_corpora/Jane_Austen/')

austen_texts = []

for filename in os.listdir('./'):
    text_data = open(filename).read().replace('\n', ' ')
    austen_texts.append(text_data)

print(len(austen_texts))

In [ ]:
## Removing the last novel from each list so we can use it to test our classifier

melville_train_texts = melville_texts[:-1]
austen_train_texts = austen_texts[:-1]

melville_test_text = melville_texts[-1]
austen_test_text = austen_texts[-1]

In [ ]:
## Creating a master list of Melville sentences

from nltk.tokenize import sent_tokenize

melville_combined_texts = ' '.join(melville_train_texts)

melville_sentences = sent_tokenize(melville_combined_texts)

print(len(melville_sentences))

melville_sentences[9999]

In [ ]:
## Extracting 2000 Melville sentences at random for use as a training set

import random

melville_train_sentences = random.sample(melville_sentences, 2000)

In [ ]:
## Creating a list of Melville sentences for our test set

melville_test_sentences = sent_tokenize(melville_test_text)

print(len(melville_test_sentences))

melville_test_sentences[997]

In [ ]:
## Creating a master list of Austen sentences

austen_combined_texts = ' '.join(austen_train_texts)

austen_sentences = sent_tokenize(austen_combined_texts)

print(len(austen_sentences))

austen_sentences[8979]

In [ ]:
## Extracting 2000 Austen sentences at random for use as a training set

austen_train_sentences = random.sample(austen_sentences, 2000)

In [ ]:
## Creating a list of Austen sentences for our test set

austen_test_sentences = sent_tokenize(austen_test_text)

print(len(austen_test_sentences))

austen_test_sentences[1000]

In [ ]:
## Combing training data
combined_texts = melville_train_sentences + austen_train_sentences

## Creating list of associated class values: 
## 0 for Melville, 1 for Austen
y = [0]*len(melville_train_sentences) + [1]*len(austen_train_sentences)

In [ ]:
## Creating vectorized training set using our combined sentence list

from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer()

X = vectorizer.fit_transform(combined_texts).toarray()

X.shape

In [ ]:
## Training a multinomial naive Bayes classifier
## X is a combined list of Melville and Austen sentences (2000 sentences from each)
## y is a list of classes (0 or 1)

from sklearn.naive_bayes import MultinomialNB

classifier = MultinomialNB().fit(X, y)

In [ ]:
## Classifying 5 sentences in our Austen test set
# Recall that 0 means Melville & 1 means Austen

from pprint import pprint

input_sentences = austen_test_sentences[3000:3005]

input_vector = vectorizer.transform(input_sentences)   ## Converting a list of string to the same
                                                       ## vector format we used for our training set.
pprint(austen_test_sentences[3000:3005])

classifier.predict(input_vector)

In [ ]:
## Classifying 5 sentences in our Melville test set

input_sentences = melville_test_sentences[3000:3005]

input_vector = vectorizer.transform(input_sentences)

pprint(melville_test_sentences[3000:3005])

classifier.predict(input_vector)

### ▷Assignment

    Write a script that prints Austen-like sentences written 
    by Melville, and Melville-like sentences written by Austen.